## Exercise-2

Wenfeng Zhu-Computer Vision and Deep Learning: Visual Synthesis

### Task-1: Linear Least Squares

1. **Derive the following formula for the minimizer w∗ of L**

As shown in the task description, the loss function of w is:
$$
L(w)= \frac 12||X_w-y||_2^2
$$
So we have to take the first order partial derivative of the function and make the partial derivative 0.\
And its corresponding norm operation can be transformed into:
$$
L(w)= \frac 12(Xw-y)^T(Xw-y)
$$
The derivative of this equation and make it to be 0:
$$
\frac {d}{dw} L(w)= X^T(Xw-y)=0
$$
Then we get:
$$
X^TXw=X^Ty
$$
We know that $ X^TX $ is  square matrix and its multiplication by its own inverse matrix is the unit matrix, so we can conclude that:
$$
w^* = (X^TX)^{-1}X^Ty
$$

2. **Implement the computation of w∗ in the fit method of LinearLeastSquares in the file task1.py.**